In [2]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

In [31]:
lebron = pd.read_csv("./data/lebron_merged_data.csv")
print(lebron.columns)

Index(['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', 'Home',
       'DRtg', 'Pace', 'eFG%', 'TOV%', 'DRB%', 'FT/FGA', 'PTS_last_5_avg',
       'FGA_last_5_avg', 'MP_last_5_avg', 'Opp_DRtg_x_PTS', 'Opp_Pace_x_FGA',
       'Opp_eFG_x_PTS'],
      dtype='object')


In [14]:
print(lebron.columns)
lebron['PTS_last_5_avg'] = lebron['PTS'].rolling(window=5, min_periods=1).mean()
lebron['FGA_last_5_avg'] = lebron['FGA'].rolling(window=5, min_periods=1).mean()
lebron['MP_last_5_avg'] = lebron['MP'].rolling(window=5, min_periods=1).mean()
lebron['Opp_DRtg_x_PTS'] = lebron['DRtg'] * lebron['PTS_last_5_avg']
lebron['Opp_Pace_x_FGA'] = lebron['Pace'] * lebron['FGA_last_5_avg']
lebron['Opp_eFG_x_PTS'] = lebron['eFG%'] * lebron['PTS_last_5_avg']
# lebron['Date'] = pd.to_datetime(df['Date'])


Index(['Unnamed: 0.1', 'Unnamed: 0', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'GmSc', 'Home', 'DRtg', 'Pace', 'eFG%', 'TOV%',
       'DRB%', 'FT/FGA', 'Unnamed: 21', 'PTS_last_5_avg', 'FGA_last_5_avg',
       'MP_last_5_avg', 'Opp_DRtg_x_PTS', 'Opp_Pace_x_FGA', 'Opp_eFG_x_PTS'],
      dtype='object')


In [32]:
scaler = StandardScaler()
to_scale = ['Home', 'DRtg', 'Pace', 'eFG%', 'TOV%', 'DRB%', 'FT/FGA', 'PTS_last_5_avg', 'FGA_last_5_avg', 'MP_last_5_avg',
        'Opp_DRtg_x_PTS', 'Opp_Pace_x_FGA', 'Opp_eFG_x_PTS']
# Apply scaling to the numerical columns
lebron[to_scale] = scaler.fit_transform(lebron[to_scale])


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

to_drop = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc'
       ]

# Define features (X) and target (y)
y = lebron['PTS']  # Target variable: Points scored

# Drop unnecessary columns (like game date, player name, etc.)
X = lebron.drop(columns=to_drop, errors='ignore')  # Keep only useful predictors




In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X.columns)


Index(['Home', 'DRtg', 'Pace', 'eFG%', 'TOV%', 'DRB%', 'FT/FGA',
       'PTS_last_5_avg', 'FGA_last_5_avg', 'MP_last_5_avg', 'Opp_DRtg_x_PTS',
       'Opp_Pace_x_FGA', 'Opp_eFG_x_PTS'],
      dtype='object')


In [36]:
# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [37]:
# Predict on test set
y_pred = model.predict(X_test)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Mean Absolute Error (MAE): 3.85
Root Mean Squared Error (RMSE): 4.48


In [38]:
# Example input for a future game
future_game = pd.DataFrame({
    'Home': [0],  # 1 for home, 0 for away
    'DRtg': [112.6],  # Example defensive rating of the opponent
    'Pace': [99.8],  # Example pace
    'eFG%': [0.528],  # Effective FG% allowed by the opponent
    'TOV%': [11.4],  # Turnover % forced
    'DRB%': [76.7],  # Defensive rebound %
    'FT/FGA': [0.179],  # Free throws allowed per FGA
    'GmSc', 
    'PTS_last_5_avg', 
    'FGA_last_5_avg', 
    'MP_last_5_avg', 
    'Opp_DRtg_x_PTS', 
    'Opp_Pace_x_FGA', 'Opp_eFG_x_PTS'
})

scaler = joblib.load("scaler.pkl")
# Ensure it matches the training features
future_game = future_game[X.columns]
future_game_scaled = scaler.transform(future_game)
# Make prediction
predicted_pts = model.predict(future_game_scaled)[0]
print(f"Predicted Points: {predicted_pts:.2f}")


SyntaxError: ':' expected after dictionary key (2513462059.py, line 10)

In [ ]:


# Load data
player_df = lebron
team_stats_df = pd.read_csv("./data/TEAM MISC STATS.csv")

print(team_stats_df.columns)
team_stats_df = team_stats_df.drop(columns=['Team','W','L','PW','PL','MOV','SOS','SRS','ORtg','FTr','3PAr','eFG%','TOV%','ORB%',,'SOS','SRS','ORtg','FTr', '3PAr', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA'])
team_stats_df.rename(columns={'eFG%.1': 'eFG%', 'TOV%.1': 'TOV%', 'FT/FGA.1':'FT/FGA'}, inplace=True)
team_stats_df.to_csv("./data/TEAM MISC STATS.csv", index=False)

In [ ]:
# Select the game to predict
game = player_df.loc[player_df['Date'] == '2025-01-15'].iloc[0]
opponent = game['Opponent']

# Get team stats for that opponent on the same day (or closest date)
team_stats = team_stats_df[(team_stats_df['Team'] == opponent) & (team_stats_df['Date'] == '2025-01-15')].iloc[0]

# Construct input row
input_row = pd.DataFrame({
    'Home': [1 if game['Location'] == 'Home' else 0],
    'DRtg': [team_stats['DRtg']],
    'Pace': [team_stats['Pace']],
    'eFG%': [team_stats['eFG%']],
    'TOV%': [team_stats['TOV%']],
    'DRB%': [team_stats['DRB%']],
    'FT/FGA': [team_stats['FT/FGA']],
    'GmSc': [game['GmSc']],
    'PTS_last_5_avg': [player_df[player_df['Date'] < game['Date']].tail(5)['PTS'].mean()],
    'FGA_last_5_avg': [player_df[player_df['Date'] < game['Date']].tail(5)['FGA'].mean()],
    'MP_last_5_avg': [player_df[player_df['Date'] < game['Date']].tail(5)['MP'].mean()],
    'Opp_DRtg_x_PTS': [team_stats['DRtg'] * game['PTS']],
    'Opp_Pace_x_FGA': [team_stats['Pace'] * game['FGA']],
    'Opp_eFG_x_PTS': [team_stats['eFG%'] * game['PTS']]
})

# Scale and predict
future_game = input_row[X.columns]  # Ensure correct feature order
future_game_scaled = scaler.transform(future_game)
predicted_pts = model.predict(future_game_scaled)[0]

print(f"Predicted Points: {predicted_pts:.2f}")